<a href="https://colab.research.google.com/github/yellowlarva-sketch/aml-fraud-detection/blob/main/01_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
main_dir = '/content/'
input_dir = main_dir + 'model/00_inputs/'
script_dir = main_dir + 'model/01_scripts/'
output_dir = main_dir + 'model/02_outputs/'
non_predictive_cols = ['target', 'date']

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
import os
os.chdir(script_dir)

In [9]:
id_col = 'account_id'
target = 'target'

In [16]:
df_origin = pd.read_csv(input_dir + '/HI-Small_Trans.csv')
print(df_origin.shape)
df_origin.head()

(4541372, 11)


,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:20,10,8000EBD30,10,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0.0
1,2022/09/01 00:20,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0.0
2,2022/09/01 00:00,3209,8000F4670,3209,8000F4670,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0.0
3,2022/09/01 00:02,12,8000F5030,12,8000F5030,2806.97,US Dollar,2806.97,US Dollar,Reinvestment,0.0
4,2022/09/01 00:06,10,8000F5200,10,8000F5200,36682.97,US Dollar,36682.97,US Dollar,Reinvestment,0.0
